In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["kaggle_username"] = userdata.get('kaggle_username')
os.environ["Kaggle_key"] = userdata.get('Kaggle_key')

In [ ]:
import tensorflow as tf

In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

In [ ]:
import keras
import keras_nlp

In [ ]:
# import json
# data = []
# with open("conversational_data (1)_yourgpt.jsonl") as file:
#     for line in file:
#         features = json.loads(line)
#         # Ensure it checks for "User Query" and "Bot Response" since there is no "context"
#         if "User Query" not in features or "Bot Response" not in features:
#             continue
#         # Format the example in 'user_query' and 'bot_response' format
#         template = "user_query: {user_query}\nbot_response: {bot_response}"
#         data.append(template.format(user_query=features["User Query"], bot_response=features["Bot Response"]))



# # Now 'data' contains the formatted conversation examples
# print(data[:5])  # Print first 5 examples for verification

In [ ]:
import json

data = []
# List of your JSONL files
jsonl_files = [
    "insomnia.jsonl",
    "narcolepsy.jsonl",
    "sleep apnea.jsonl",
    "sleep hygiene.jsonl",
    "sleepbooks.jsonl"
]

# Loop through each file and process it
for jsonl_file in jsonl_files:
    with open(jsonl_file) as file:
        for line in file:
            features = json.loads(line)
            # Ensure it checks for "User Query" and "Bot Response" since there is no "context"
            if "User Query" not in features or "Bot Response" not in features:
                continue
            # Format the example in 'user_query' and 'bot_response' format
            template = "user_query: {user_query}\nbot_response: {bot_response}"
            data.append(template.format(user_query=features["User Query"], bot_response=features["Bot Response"]))

# Now 'data' contains the formatted conversation examples from all files
print(data[:5])  # Print first 5 examples for verification


[]


In [ ]:
!mkdir -p ~/.kaggle
!echo '{"username":"sanjanakumari8113","key":"21473c2afd099f395d2571a0bce6699f"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.model_download("keras/gemma2/keras/gemma2_2b_en")

print("Path to model files:", path)

Path to model files: /kaggle/input/gemma2/keras/gemma2_2b_en/1


In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[256000,2304] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomNormalV2]

In [ ]:

# # Assuming the template follows 'user_query' and 'bot_response'
# prompt = template.format(
#     user_query="How can I use aromatherapy to improve my sleep?",
#     bot_response="",
# )

# sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
# gemma_lm.compile(sampler=sampler)
# print(gemma_lm.generate(prompt, max_length=256))


In [ ]:
# Assuming the template follows 'user_query' and 'bot_response'
template = "user_query: {user_query}\nbot_response: {bot_response}" # Define template here
prompt = template.format(
    user_query="How can I use aromatherapy to improve my sleep?",
    bot_response="",
)

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=1000))

{"User Query":"How can I use aromatherapy to improve my sleep?","Bot Response":"Essential oils like lavender and chamomile can have calming effects. Diffusing these oils in your bedroom before bedtime may help promote relaxation and better sleep."}


In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=7)
gemma_lm.summary()

In [ ]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 1000
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

In [ ]:


# ... (your previous code) ...

# Before fitting the model, ensure 'data' is a tf.data.Dataset with string elements:
def ensure_string_dataset(dataset):
  """Converts dataset elements to strings if they are not already."""
  def to_string(element):
    if not isinstance(element, tf.Tensor) or element.dtype != tf.string:
      return tf.strings.as_string(element)
    return element
  return dataset.map(to_string)

data = ensure_string_dataset(data)

# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 1000
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

gemma_lm.fit(data, epochs=1, batch_size=1)

In [ ]:

# # Assuming the template follows 'user_query' and 'bot_response'
# prompt = template.format(
#     user_query="What should I do on a trip to Europe?",
#     bot_response="",
# )

# sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
# gemma_lm.compile(sampler=sampler)
# print(gemma_lm.generate(prompt, max_length=256))


In [ ]:

# Assuming the template follows 'user_query' and 'bot_response'
prompt = template.format(
    user_query="I have difficulty falling asleep. What are some effective strategies to help with this?",
    bot_response="",
)

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))


In [ ]:

# Assuming the template follows 'user_query' and 'bot_response'
prompt = template.format(
    user_query="How can I use aromatherapy to improve my sleep?",
    bot_response="",
)

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))


In [ ]:
# Define a custom callback to print the accuracy during training
class AccuracyCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        accuracy = logs.get('sparse_categorical_accuracy')
        if accuracy is not None:
            print(f'Accuracy after epoch {epoch + 1}: {accuracy:.4f}')

# Create an instance of the callback
accuracy_callback = AccuracyCallback()

# Pass the callback to the fit method
gemma_lm.fit(data, epochs=1, batch_size=1, callbacks=[accuracy_callback])